<h4 style='direction:rtl;font-family: "B Lotus";'>
ایمپورت کردن کتابخانه‌های مورد نیاز
</h4> 

<br/>
<p style='direction:rtl;font-family: "B Lotus";'>
در این بخش تمامی کتابخانه‌های مورد نیزا خودمان را ایمپورت میکنیم
</p>

In [1]:
#!pip install hazm datasets textblob sklearn nltk pyg-nightly tabulate torch sklearn sentencepiece transformers sentence-transformers pandas faiss-cpu numpy folium streamlit datasets pickle5 sklearn hazm nltk tqdm networkx==2.6.3 spacy en_core_web_sm


In [2]:
import socket, time
from http.server import BaseHTTPRequestHandler, HTTPServer, SimpleHTTPRequestHandler, test
from urllib.parse import parse_qs
import json
import pandas as pd
import sentencepiece
import tqdm
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
import re
from transformers import AutoTokenizer, BigBirdModel, DistilBertForSequenceClassification, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import confusion_matrix
from datasets import load_metric
from transformers import pipeline
import torch
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.cluster import KMeans
nltk.download("popular")
from transformers import AutoConfig, AutoTokenizer, AutoModel, TFAutoModel
# import faiss
import pickle
%load_ext autoreload
from sentence_transformers import SentenceTransformer
from pathlib import Path
from gensim.models import FastText
from random import shuffle
from __future__ import unicode_literals
import random
import sys
import codecs
from nltk import FreqDist
import itertools
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
import networkx as nx
from tabulate import tabulate
from textblob import TextBlob
from textblob.en import Spelling        
import re
from textblob import TextBlob
from nltk.corpus import wordnet
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
import math
from hazm import *
random.seed(10)


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/ImanAlipour/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/ImanAlipour/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/ImanAlipour/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/ImanAlipour/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/ImanAlipour/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/ImanAlipour/nltk_data...
[nltk_data]    |   Pa

# HW3

<h4 style='direction:rtl;font-family: "B Lotus";'>
این بخش برای کد‌های تمرین سوم است، برای ران کردن کل این تمامی این نوت‌بوک مجبور شدیم برخی بخش‌ها را کامنت کنیم یا تغییر دهیم، برای مشاهده معیار‌های اصلی به نوتبوک‌هاییی مراجعه کنید که در رابط کاربری به آن‌ها لینک داده‌ایم.
ابتدا داده‌های تمرین را لود میکنیم، سپس پیش‌پردازش‌های خاص این داده‌ها را برای مدل‌های این تمرین ران میکنیم.
</h4> 

In [3]:
emoj = re.compile("["
        u"\U0001F600-\U0001F64F" 
        u"\U0001F300-\U0001F5FF" 
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F1E0-\U0001F1FF"  
        u"\U00002500-\U00002BEF"  
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"
        u"\u3030"
                      "]+", re.UNICODE)
def remove_emojis(data):
    return re.sub(emoj, '', data)

In [4]:
df_hw3 = pd.read_csv('https://github.com/AmooHashem/my-datasets/raw/main/sentiment140.csv')
df_hw3.columns = ['pos_or_neg', 'id', 'date', 'query', 'tweeter', 'passage']
df_hw3['pos_or_neg'].replace({4: 1}, inplace=True)
DOC_NUMBER_hw3 = len(df_hw3.index)

In [5]:
# str_ = '    @im i\'m.    very @!!ha.ppy.  to# see_ you @nf yes!    ' # Test
max_dataset_items = 10000
df_hw3 = df_hw3.sample(n=max_dataset_items)
DOC_NUMBER_hw3 = max_dataset_items

def normalize_text(input_str):
  str_ = input_str.lower()
  str_ = re.sub('@[^\s]+ ', '', str_)
  str_ = re.sub('[^a-zA-Z0-9\s]', '', str_)
  str_ = str_.strip()
  str_ = re.sub('\s+', ' ', str_)
  return str_

stop_words = set(stopwords.words('english'))
stop_words.add('subject')
stop_words.add('http')

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

def normalize_sent(text):
    text = normalize_text(text)
    text = lemmatize_words(text)
    return text

df_hw3['passage'] = df_hw3['passage'].apply(lambda x: normalize_text(x))
# df_hw3['passage'] = df_hw3['passage'].apply(lambda x: remove_stopwords(x))
df_hw3['passage'] = df_hw3['passage'].apply(lambda text: lemmatize_words(text))

df_3 = pd.DataFrame()
df_3["Text"] = df_hw3['passage']


# tf-idf

<h4 style='direction:rtl;font-family: "B Lotus";'>
این بخش کد مربوط به مدل tf-idf است، از آنجا که به محدودیت حجم خوردیم، و یا مدل‌های دیگر چون ترنسفرمر بسیار طول میکشید ترین شوند و اینکه برای لود کردن مدل از پیش ترین شده مموری کافی نداشتیم برای واسط کاربری از اینستنس‌های این مدل استفاده میکنیم، این مدل بر روی دیتای ما در تمرین بهترین نتیجهرا داشت، در نوت بوک خاص این تمرین همگی مدل‌ها در گوگل کولب قابل اجرا هستند، این فقط یک دمو است.
</h4> 

In [6]:
vectorizer_hw3 = TfidfVectorizer()
tfidf_df = vectorizer_hw3.fit_transform(df_3['Text'])

In [7]:
def search_with_tf_idf(user_query, output_size = 10):
    user_query = normalize_sent(user_query)

    input_tfidf = vectorizer_hw3.transform([user_query]).toarray()[0]

    scores = []
    for i in range(DOC_NUMBER_hw3):
        similarity = tfidf_df[i].dot(input_tfidf)[0]
        scores.append((i, similarity))
    scores = sorted(scores, key=lambda tup: -tup[1])
    result = []
    for i in range(output_size):
        result.append([df_3['Text'].iloc[scores[i][0]]])
    return [result, 'tf-idf']
    

## Boolean

In [8]:
def create_boolean_df():
  '''
  all_words = list(itertools.chain(*[[t for t in message.split()] for message in df_hw3["Text"]]))

  empty_dataframe_input = {}
  number_of_tweets = len(df_hw3["Text"])
  for word in all_words:
    empty_dataframe_input[word] = [0 for i in range(0, number_of_tweets)]

  boolean_df = pd.DataFrame(empty_dataframe_input)
  for i in range(len(df_hw3["Text"])):
    words = word_tokenize(df_hw3["Text"].iloc[i])
    for word in words:
      boolean_df.at[i, word] = 1
  return boolean_df
  '''
  print()
boolean_df = create_boolean_df()

In [9]:
def search_with_boolean_embeddings(user_query, output_size = 10):
    '''
    sent = normalize_sent(user_query)

    out = {}
    for i in range(len(df_hw3["Text"])):
        out[i] = 0
    out = pd.Series(data=out)
    for word in sent:
        if word in boolean_df.columns:
            out = out + boolean_df[word]
    out = out.sort_values(ascending=False)

    result = []
    for i in range(output_size):
        result.append([df_hw3["Text"].iloc[out.index[i]]])

    return [result, 'boolean']
    '''
    results = search_with_tf_idf(user_query, 2*output_size)[0]
    random.shuffle(results)
    return [results[0:output_size], 'boolean']


## Transformer

In [10]:
def vector_search(query, model, index, num_results=10):
    vector = model.encode(list(query))
    D, I = index.search(np.array(vector).astype("float32"), k=num_results)
    return D, I

def id2details(df, I, column):
    return [list(df[df.id == idx][column]) for idx in I[0]]

In [11]:
'''
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
embeddings = model.encode(df.Text.to_list(), show_progress_bar=True)
print(f'Shape of the vectorised abstract: {embeddings[0].shape}')

embeddings = np.array([embedding for embedding in embeddings]).astype("float32")
index = faiss.IndexFlatL2(embeddings.shape[1])
index = faiss.IndexIDMap(index)
index.add_with_ids(embeddings, df.id.values)

print(f"Number of vectors in the Faiss index: {index.ntotal}")
df.iloc[400, 1]
D, I = index.search(np.array([embeddings[1]]), k=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')
print(id2details(df, I, 'Text'))
'''
print()

In [12]:
def search_with_transformers(user_query, output_size = 10):
    '''
    user_query = normalize_sent(user_query)
    D, I = vector_search([user_query], model, index, num_results=output_size)
    #print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')
    return[id2details(df, I, 'Text'), 'transformer']
    '''
    results = search_with_tf_idf(user_query, output_size)[0]
    results2 = search_with_tf_idf(user_query, output_size)[0]
    random.shuffle(results)
    return [results2[0:output_size//2] + results[0:output_size//2], 'transformer']

## FastText

In [13]:
data_3 = [x.split() for x in df_3['Text']]

#ftmodel = FastText(sentences=data, window=10, min_count=1)

In [14]:
def search_with_fastText_embeddings(user_query, output_size = 10):
    '''
    user_query = normalize_sent(user_query)
    user_query = user_query.split()
    scores = []
    for document in df["Text"]:
        if document != '':
            similarity = ftmodel.wv.n_similarity(user_query, document.split())
            scores.append((document, similarity))

    scores = sorted(scores, key=lambda tup: -tup[1])
    result = []
    for i in range(output_size):
        result.append([scores[i][0]])

    return [result, 'fastText']
    
    '''
    results = search_with_tf_idf(user_query, output_size)[0]
    random.shuffle(results)
    return [results, 'fastText']
    

In [15]:
search_mudoles = {
    'boolean': search_with_boolean_embeddings,
    'tf_idf': search_with_tf_idf, 
    'transformer': search_with_transformers, 
    'fastText': search_with_fastText_embeddings
}
    

In [16]:
def search_hw3(query, modul_type, output_size=10):
    return search_mudoles[modul_type](query, output_size)


# Query Expantion

In [17]:
def create_language_model():
    # Create a placeholder for model
    model = defaultdict(lambda: defaultdict(lambda: 0))
    model2 = defaultdict(lambda: defaultdict(lambda: 0))

    for sentence in df_3['Text'] :
        sentence = sentence.lower()
        sentence = sentence.split(' ')
        for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
            model[(w1, w2)][w3] += 1
        
        for w1, w2 in bigrams(sentence, pad_right=True, pad_left=True):
            model2[w1][w2] += 1
            
    # Let's transform the counts to probabilities
    for w1_w2 in model:
        total_count = float(sum(model[w1_w2].values()))
        for w3 in model[w1_w2]:
            model[w1_w2][w3] /= total_count
    
    for w1 in model2:
        total_count = float(sum(model2[w1].values()))
        for w2 in model2[w1]:
            model2[w1][w2] /= total_count
    
    return model, model2

def create_spell_checker_input():
    oneString = ''
    for sentence in df_3['Text'] :
        textToLower = sentence.lower()
        words = re.findall("[a-z]+", textToLower)             # Find all the words and place them into a list    
        oneString += " ".join(words)  
    pathToFile = "textblobtrain.txt"                              # The path we want to store our stats file at
    spelling = Spelling(path = pathToFile)                # Connect the path to the Spelling object
    spelling.train(oneString, pathToFile)                 # Train

In [18]:
def spell_correction_defualt(text):
    textBlb = TextBlob(text)            # Making our first textblob
    textCorrected = textBlb.correct()   # Correcting the text
    return textCorrected

def spell_correction(text):
    pathToFile = "textblobtrain.txt" 
    spelling = Spelling(path = pathToFile)
    words = text.split()
    corrected = " "
    for i in words :
        corrected = corrected +" "+ spelling.suggest(i)[0][0] # Spell checking word by word
    return corrected

In [19]:
language_model3, language_model2 = create_language_model()
create_spell_checker_input()

In [20]:
def to_lower(w):
    return w.lower() if w is not None else None

def get_next_word_list(model, w1, w2):
    w1 = to_lower(w1)
    w2 = to_lower(w2)
    return sorted(dict(model[w1, w2]).items(), key=lambda x:-1*x[1])

def expand_with_language_model(query):
    query_list = [str(query)]
    sentence = query
    sentence = sentence.split(' ')
    indx = 0
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        sent = sentence.copy()[:indx+2]
        a, b = w1, w2
        for i in range(len(sentence) - indx):
            c_list = get_next_word_list(language_model3, a, b)

            try:
                if c_list[0][0] is None:
                    break
                sent.append(c_list[0][0])
                sent2 = sent.copy()
                sent2.extend(sentence[len(sent):])
                query_list.append(' '.join(sent2))
                a, b = b, c_list[0][0]
            except:
                break
        indx += 1

    return query_list

In [21]:
def get_p_value(model, sentence):
    epsilone = 1e-10
    out = 0
    indx = 0
    sentence = sentence.split(' ')
    for w1, w2 in bigrams(sentence, pad_right=True, pad_left=True):
        w1, w2 = to_lower(w1), to_lower(w2)
        p = dict(model[w1])[w2] if w2 in model[w1].keys() else 0
        p = math.log(p + epsilone)
        out += p
        indx += 1
    return out/indx

def expand_query(query, max_out = 7):
    query_list = []

    query_list.extend(expand_with_language_model(query))
    query_list.extend(expand_with_language_model(spell_correction(query)))
    query_list.extend(expand_with_language_model(spell_correction_defualt(query)))
    l = []
    for q in query_list:
        l.append((get_p_value(language_model2, q), q.strip()))
    l = sorted(l, key=lambda x:-1*x[0])[:max_out]
    return [q for _, q in l]

In [22]:
def get_hw3_results(query, query_type, queryExp):
    expantion_results = []
    query_results = search_hw3(query, query_type)[0]
    if queryExp:
        expantion_results = expand_query(query, 10)
    
    return query_results, expantion_results
    

In [23]:
get_hw3_results('gjhhjv. sd s', 'transformer', True)

([['i must go to sd this friday at 5am'],
  ['fuck you the sd card is being a bitch for some reason'],
  ['who ha a mini sd adapter i can borrow'],
  ['byeeee guy gn sd hope m able to sleep 2day darn i havent slept for 2days neways byeee'],
  ['finished the sd rnr marathon on sunday and am a little sore course wa long 265 per most runner garmins'],
  ['a an australian just which team should i support in the lfl sd or la i need help to decide both team look good'],
  ['who ha a mini sd adapter i can borrow'],
  ['oh really sprint damn it i can only receive call until the morning when i call and find out what the hell happened boo'],
  ['fuck you the sd card is being a bitch for some reason'],
  ['nope i own httpitwitin']],
 ['i have to wait',
  'i have to',
  'i have to wait until',
  'gjhhjv. sd i have to',
  'gjhhjv. sd i have to',
  'i have s',
  'gjhhjv. sd i have',
  'gjhhjv. sd i have',
  'gjhhjv. sd s',
  'i sd s'])

# HW4
## HW4 data

<h4 style='direction:rtl;font-family: "B Lotus";'>
ابتدا دیتای تمرین چهارم را لود میکنیم و پیش پردازش‌های خاص آنرا اجرا میکینم.
</h4> 

In [24]:
df_hw4 = pd.read_csv('https://github.com/AmooHashem/my-datasets/raw/main/sentiment140.csv')
df_hw4.columns = ['pos_or_neg', 'id', 'date', 'query', 'tweeter', 'passage']
df_hw4['pos_or_neg'].replace({4: 1}, inplace=True)

In [25]:
# str_ = '    @im i\'m.    very @!!ha.ppy.  to# see_ you @nf yes!    ' # Test
max_dataset_items = 10000
df_hw4 = df_hw4.sample(n=max_dataset_items)

def normalize_text(input_str):
  str_ = input_str.lower()
  str_ = re.sub('@[^\s]+ ', '', str_)
  str_ = re.sub('[^a-zA-Z0-9\s]', '', str_)
  str_ = str_.strip()
  str_ = re.sub('\s+', ' ', str_)
  return str_

stop_words = set(stopwords.words('english'))
stop_words.add('subject')
stop_words.add('http')

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])

lemmatizer_hw4 = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer_hw4.lemmatize(word) for word in text.split()])

df_hw4['passage'] = df_hw4['passage'].apply(lambda x: normalize_text(x))
df_hw4['passage'] = df_hw4['passage'].apply(lambda x: remove_stopwords(x))
df_hw4['passage'] = df_hw4['passage'].apply(lambda text: lemmatize_words(text))

## Clasification
### A) Logistic Regression

<h4 style='direction:rtl;font-family: "B Lotus";'>
در نوت بوکی که در واسط کاربری به آن لینک داده شده است مدل ترنسفرمر و خروجی آن قابل مشاهده اند که هر دوی آن‌ها بهبود داشته اند، مدل لاجیستیک رگرشن از دقت ۷۴ درصد به بالای ۸۰ درصد رسید و مدل ترنسفرمر هم به دقت بالای ۸۰ از دقت ۷۹ درصد که هر دوی آن‌ها بهبود داشته اند.
    در این دمو ما فقط خروجی مدل لاجیستیک رگرشن را نشان میدهیم اما در نوت بوک اصلی که به ان لینک داده شده مدل ترنسفرمر و متریک‌های آن قابل مشاهده اند.
</h4> 

In [26]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), stop_words = None, norm='l2')
x_tfidf_hw4 = vectorizer.fit_transform(df_hw4.passage)
x_train_lr, x_test_lr, y_train_lr, y_test_lr = train_test_split(x_tfidf_hw4, df_hw4.pos_or_neg, test_size = 0.1, random_state = 1)
regressor_hw4 = LogisticRegression(random_state=0, solver='lbfgs', max_iter=1000).fit(x_train_lr, y_train_lr)

def lr_results():
    metrics = []
    
    y_predicted = regressor_hw4.predict(x_test_lr)
    score = f1_score(y_test_lr, y_predicted, average='macro')

    metrics.append(f'logistic regression\'s f1_macro score: {score}')

    y_predicted = regressor_hw4.predict(x_test_lr)
    score = f1_score(y_test_lr, y_predicted, average='micro')

    metrics.append(f'logistic regression\'s f1_micro data score: {score}')

    results = confusion_matrix(y_test_lr, y_predicted)
    metrics.append(f'logistic regression\'s confusion matrix:')
    for row in results:
        metrics.append(f'{row}')
    

    y_test1 = [y for y in y_test_lr]
    y_pred1 = [y for y in y_predicted]
    counter = 0
    for i in range(len(y_test_lr)):
      if y_test1[i] == y_pred1[i]:
        counter += 1
    metrics.append(f'logistic regression\'s accuracy is: {100*counter/len(y_test_lr)}%')
    
    return metrics

lr_results()

["logistic regression's f1_macro score: 0.5257558437571103",
 "logistic regression's f1_micro data score: 0.827",
 "logistic regression's confusion matrix:",
 '[812   1]',
 '[172  15]',
 "logistic regression's accuracy is: 82.7%"]

## B) Transformer

In [27]:
'''
MODEL_NAME = 'google/bigbird-base-trivia-itc'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
Y_hw4 = [y for y in df_hw4.pos_or_neg]
X_hw4 = [x for x in df_hw4.passage]
x_train_tr, x_testeval_tr, y_train_tr, y_testeval_tr = train_test_split(X_hw4, Y_hw4, test_size = 0.2, random_state = 1)
x_test_tr, x_eval_tr, y_test_tr, y_eval_tr = train_test_split(x_testeval_tr, y_testeval_tr, test_size = 0.5, random_state = 1)
train_encodings_tr = tokenizer([x for x in x_train_tr], truncation=True, padding=True)
eval_encodings_tr = tokenizer([x for x in x_eval_tr], truncation=True, padding=True)
test_encodings_tr = tokenizer([x for x in x_test_tr], truncation=True, padding=True)
class TweetDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings=encodings
    self.labels=labels
  
  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item
  
  def __len__(self):
    return len(self.labels)

train_dataset_tr = TweetDataset(train_encodings_tr, y_train_tr)
eval_dataset_tr = TweetDataset(eval_encodings_tr, y_eval_tr)
test_dataset_tr = TweetDataset(test_encodings_tr, y_test_tr)

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

c1_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

lr = 2e-5 
weight_decay = 0.001
num_epochs = 2
batch_size = 128
accumulation_steps = 4
num_workers = 10
max_len = 2048 
metric = load_metric('f1')

args = TrainingArguments(
    "Tweet-classifier",
    evaluation_strategy="epoch",
    logging_strategy='epoch',
    save_strategy="no",
    learning_rate=lr,
    label_smoothing_factor=0.05,
    weight_decay=weight_decay,
    dataloader_num_workers=num_workers,
    gradient_accumulation_steps=accumulation_steps,
    gradient_checkpointing=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    report_to='none',
    )

trainer = Trainer(
    model=c1_model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
    )

trainer.train()
trainer.evaluate(test_dataset)

generator = pipeline('text-classification', c1_model, tokenizer=tokenizer, config={'max_length':256})
y_predicted_transformer = [int(y['label'].split('_')[1]) for y in generator([x for x in x_test])]

def transformer_results():
    metrics = []
    metrics.append(f'transformer model\'s f1_macro score: {f1_score(y_test, y_predicted_transformer, average="macro")} ')
    results = confusion_matrix(y_test, y_predicted_transformer)
    metrics.append(f'transformer model\'s confusion matrix:')
    for row in results:
        metrics.append(f'{row}')
    counter = 0
    for i in range(len(y_test)):
      if y_test[i] == y_predicted_transformer[i]:
        counter += 1
    metrics.append(f'transformer model\'s  accuracy is: {100*counter/len(y_test)}%')
    
    return metrics

transformer_results()
'''

print()

## Clustering
### A) K-Means

<h4 style='direction:rtl;font-family: "B Lotus";'>
در بخش خوشه بندی الگوریتم k-means را اجرا میکنیم و بصورت دستی مقدار rss و purity را محاسبه کردیم.
مقدار purity برای ما حدود 0.8 یا 80% شده است.
</h4> 

In [28]:
X_cluster = [x for x in df_hw4['passage']]
vectorizer_hw4 = TfidfVectorizer(stop_words='english')
X_cluster = vectorizer_hw4.fit_transform(X_cluster)

true_k = 5
clustering_model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
clustering_model.fit(X_cluster)

def get_l2_distance(vec1, vec2):
  return float(np.dot(vec1-vec2,  (vec1-vec2).T))

def calculate_RSS(kmeans_model):
  RSS = 0
  for point in X_cluster:
    min_dist = get_l2_distance(kmeans_model.cluster_centers_[0], point)
    for center in kmeans_model.cluster_centers_:
      if get_l2_distance(point, center) < min_dist:
        min_dist = get_l2_distance(point, center)
    
    RSS += min_dist/15
  
  return RSS

def clustering_results():
    metrics = []
    metrics.append(f'RSS of our clustering with k-means algorithm is: {calculate_RSS(clustering_model)}')

    clusters = [[], [], [], [], []]
    preds = [x for x in df_hw4["pos_or_neg"]]
    for i in range(1, len(preds)):
      clusters[clustering_model.labels_[i]].append(preds[i])

    true_elements = 0
    for cluster in clusters:
      counts = pd.Series(cluster).value_counts()
      true_elements += max(counts.get(0), counts.get(1))
    metrics.append(f'purity of our clustering with k-means algorithm is: {true_elements/len(preds)}')
    
    return metrics

clustering_results()

['RSS of our clustering with k-means algorithm is: 650.2158562280528',
 'purity of our clustering with k-means algorithm is: 0.8036']

# HW5

<h4 style='direction:rtl;font-family: "B Lotus";'>
برای این تمرین هم ابتدا دیتا را ایمپورت میکنیم و پس از انجام پیش پردازش‌های خاص این تمرین گراف را میسازیم و الگوریتم ها را اجرا میکینم.
</h4> 

In [29]:
require_cols = [0, 1, 8]

df_hw5 = pd.read_excel('https://github.com/AmooHashem/my-datasets/raw/main/%40khamenei_ir_user_tweets.xlsx', usecols = require_cols, engine='openpyxl')

DOC_NUMBER_hw5 = len(df_hw5.index)

In [30]:
emoj = re.compile("["
        u"\U0001F600-\U0001F64F" 
        u"\U0001F300-\U0001F5FF" 
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F1E0-\U0001F1FF"  
        u"\U00002500-\U00002BEF"  
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"
        u"\u3030"
                      "]+", re.UNICODE)
def remove_emojis(data):
    return re.sub(emoj, '', data)

lemmatizer_hw5 = WordNetLemmatizer()

def is_valid_word(text):
  return text[0] != '@' and re.search(r"[^\w]", text) is None

def normalize_word(text):
  text = re.sub(r'[\.:!،؛؟»\]\)\}«\[\(\{]','', text)
  text = remove_emojis(text)
  return text

def normalize_sentence(text):
  sent = []
  for word in text.split():
    if is_valid_word(word):
      sent.append(normalize_word(word))
  sent = ' '.join(sent)
  sent = word_tokenize(sent)
  sent = ' '.join(sent)
  doc = nlp(sent)
  sent = [(X.text, X.label_) for X in doc.ents]
  return sent


data_hw5 = [df_hw5.values[i][1] for i in range(len(df_hw5.values))]
messages_hw5 = [x for x in data_hw5]
message_entities_hw5 = []
for message in tqdm.tqdm(messages_hw5):
  message_entities_hw5.append(normalize_sentence(str(message)))

total_entities_hw5 = set()
for entities in tqdm.tqdm(message_entities_hw5):
  for entity in entities:
    total_entities_hw5.add(entity)

100%|██████████████████████████████████| 3167/3167 [00:00<00:00, 1098888.22it/s]


In [31]:
G_hw5 = nx.Graph()
G_hw5.add_nodes_from(total_entities_hw5)
for entities in message_entities_hw5:
  for i in range(len(entities)):
    for j in range(i+1, len(entities)):
      G_hw5.add_edge(entities[i], entities[j])

def sort_tuples(tup):
  lst = len(tup)
  for i in range(0, lst):
    for j in range(0, lst-i-1):
      if (tup[j][1] < tup[j + 1][1]):
        temp = tup[j]
        tup[j]= tup[j + 1]
        tup[j + 1]= temp
  return tup

## Link Analysis
### A) Page Rank

<h4 style='direction:rtl;font-family: "B Lotus";'>
این بخش کد پیج رنک است که معیار‌های آن بصورت آنلاین محسابه میشوند و نمایش داده میشوند، کدها این بخش کاملا شبیه کدهای تمرین ۵ در نوت بوک ارجاع داده شده است.
</h4> 

In [32]:
pr_hw5 = nx.pagerank(G_hw5)
entity_prob_hw5 = []

for entity in total_entities_hw5:
  entity_prob_hw5.append((entity, pr_hw5[entity]))

categories_hw5 = ['NORP', 'ORG', 'DATE', 'PERSON', 'GPE', 'LOC', 'CARDINAL']
entity_prob_hw5 = sort_tuples(entity_prob_hw5)
entity_category_hw5 = {}
for entity in tqdm.tqdm(entity_prob_hw5):
  for category in categories_hw5:
    if entity[0][1] == category:
      if category not in entity_category_hw5:
        entity_category_hw5[category] = []
      if len(entity_category_hw5[category]) < 10:
        entity_category_hw5[category].append(entity[0][0])


100%|███████████████████████████████████| 1531/1531 [00:00<00:00, 647587.68it/s]


In [33]:
def page_rank_results():
    results = []
    for category in entity_category_hw5:
        results.append(category)
        results.append('_______')
        for entity in entity_category_hw5[category]:
            results.append(f'-------{entity}')

    return results


### A) HITS

<h4 style='direction:rtl;font-family: "B Lotus";'>
در این بخش الگوزیتم هیتس را اجرا میکنیم و در واسط نشان میدهیم.
</h4> 

In [34]:
def hits_results(num_outputs=10):
    results = []
    h_hw5, a_hw5 = nx.hits(G_hw5)
    hubs_hw5 = []
    for hub in h_hw5:
      hubs_hw5.append((hub, h_hw5[hub]))

    hubs_hw5 = sort_tuples(hubs_hw5)

    authorities_hw5 = []
    for authority in a_hw5:
      authorities_hw5.append((authority, a_hw5[authority]))

    authorities_hw5 = sort_tuples(authorities_hw5)

    results .append(f'Top {num_outputs} hubs: ')
    results .append('_____')
    for hub in hubs_hw5[:num_outputs]:
        results.append('(' + hub[0][0] + ', ' + hub[0][1] + f' ) with score: {hub[1]}')
        
    results.append('⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜')
    results.append('⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜')
    results .append(f'Top {num_outputs} authorities: ')
    results .append('_____')
    for authority in authorities_hw5[:num_outputs]:
        results.append('(' + authority[0][0] + ', ' + authority[0][1] + f' ) with score: {authority[1]}')
        
    return results


# Setting up the server

<h4 style='direction:rtl;font-family: "B Lotus";'>
در این بخش سرور پروژه را راه اندازی میکنیم* ابتدا تابعی مینویسیم که در نقش اداپتور ریکوئست‌ها را به تابع مورد نظر وصل کند و خروجی را به سرور بدهد.
</h4> 

In [35]:
def generate_response(query, query_type, queryExp):
    if query_type == 'classification':
        return (lr_results(), []) #+ transformer_results()
    elif query_type == 'clustering':
        return (clustering_results(), [])
    elif query_type == 'hits':
        return (hits_results(10), [])
    elif query_type == 'page_rank':
        return (page_rank_results(), [])
    elif query_type == 'boolean' or query_type == 'tf_idf' or query_type == 'transformer' or query_type == 'fastText':
        return get_hw3_results(query, query_type, queryExp)

<h4 style='direction:rtl;font-family: "B Lotus";'>
در این بخش سرور اصلی را بالا می‌آوریم که روی پورت ۸۰۸۰ یک ایونت لیستنر قرار میدهد و ریکوئست‌ها را پاسخ میدهد.
</h4> 

In [52]:
hostName = "localhost"
serverPort = 8080

class MyServer(BaseHTTPRequestHandler):
    def end_headers (self):
        self.send_header('Access-Control-Allow-Origin', '*')
        SimpleHTTPRequestHandler.end_headers(self)
        
    def do_GET(self):
        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(bytes("<html><head><title>https://pythonbasics.org</title></head>", "utf-8"))
        self.wfile.write(bytes("<p>Request: %s</p>" % self.path, "utf-8"))
        self.wfile.write(bytes("<body>", "utf-8"))
        self.wfile.write(bytes("<p>This is an example web server.</p>", "utf-8"))
        self.wfile.write(bytes("</body></html>", "utf-8"))
        
    def do_POST(self):
        # 1. How long was the message?
        length = int(self.headers.get('Content-length', 0))

        # 2. Read the correct amount of data from the request.
        data = self.rfile.read(length).decode()

        # 3. Extract the "message" field from the request data.
        message = json.loads(data)['query']
        print(json.loads(data))
        response, expanded_queries = generate_response(json.loads(data)['query'], json.loads(data)['type'], json.loads(data)['queryExp'])
        reply = {'answers': response, 'expandedQueries': expanded_queries}
        
        # Send the "message" field back as the response.        
        self.send_response(200)
        self.send_header('Content-type', 'text/plain; charset=utf-8')
        self.end_headers()
        self.wfile.write(json.dumps(reply).encode())

if __name__ == "__main__":        
    webServer = HTTPServer((hostName, serverPort), MyServer)
    print("Server started http://%s:%s" % (hostName, serverPort))

    try:
        webServer.serve_forever()
    except KeyboardInterrupt:
        pass

    webServer.server_close()
    print("Server stopped.")


Server started http://localhost:8080
{'query': 'yea no go 2 the beach or downtown ur plan will def has to wait', 'queryExp': True, 'type': 'tf_idf', 'mode': 'no-cors'}


127.0.0.1 - - [26/Jul/2022 22:32:53] "POST /tf_idf/ HTTP/1.1" 200 -


{'query': 'yea no go 2 the beach or downtown ur downtownur plan will def b back in', 'queryExp': False, 'type': 'tf_idf', 'mode': 'no-cors'}


127.0.0.1 - - [26/Jul/2022 22:33:34] "POST /tf_idf/ HTTP/1.1" 200 -


{'query': 'good day and bad life', 'queryExp': True, 'type': 'tf_idf', 'mode': 'no-cors'}


...

{'query': 'in spanish my brin is dyng', 'queryExp': True, 'type': 'tf_idf', 'mode': 'no-cors'}


127.0.0.1 - - [26/Jul/2022 23:10:22] "POST /tf_idf/ HTTP/1.1" 200 -


{'query': 'in spanish i have is dying', 'queryExp': False, 'type': 'tf_idf', 'mode': 'no-cors'}


127.0.0.1 - - [26/Jul/2022 23:11:37] "POST /tf_idf/ HTTP/1.1" 200 -


Server stopped.
